In [1]:
from tensorflow.keras.models import load_model
from glob import glob
import os
import random
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from pyeit.mesh.wrapper import PyEITAnomaly_Circle
from src.vae2D import vae_model
from src.util import (
    seq_data, 
    load_2Ddata, 
    seq_data,
)
from pyeit import mesh
from keras import backend as K
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras.models import Model
from tqdm import tqdm
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

2025-04-01 14:26:36.033239: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743517596.047084  813776 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743517596.051268  813776 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-01 14:26:36.066802: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
W0000 00:00:1743517606.267020  813776 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libr

### loading training data set

In [3]:
voltage, gamma, _ = load_2Ddata("sim_data", "sim_data_set", data_type = "sim") 

Loading simulation data...
Normalizing data...
Data loading complete!


### loading pretrained VAE model

In [ ]:
vae = vae_model()  
vae.build(input_shape=(None, 2840)) 
vae.load_weights("weights/vae2D.weights.h5")
print("Weights of VAE successfully loaded.")

### generating latent representations using encoder of pretrained VAE

In [ ]:
_, _, latent_rep = vae.encoder.predict(gamma)

### mapper training

In [ ]:
(
    v_train_seq,
    v_test_seq,
    gamma_train,
    gamma_test,
    z_train,
    z_test,
) = train_test_split(
    voltage,
    gamma,
    latent_rep,
    test_size=0.1,
    shuffle=True,
    random_state=42,
)

In [4]:
from src.lstm_mapper2D import lstm_mapper2D_model
lstm_mapper = mapper_model()
lstm_mapper.summary()
lstm_mapper.compile(optimizer="adam", loss="mse")


Model: "lstm_mapper"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 4, 32, 32, 1)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed                │ (None, 4, 15, 15, 2)   │            20 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_1              │ (None, 4, 7, 7, 2)     │            38 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_2              │ (None, 4, 98)          │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_3              │ (None, 4, 64)          │         6,336 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 4, 32)          │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 16)             │         3,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 8)              │           136 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,082 (86.26 KB)

 Trainable params: 22,082 (86.26 KB)

 Non-trainable params: 0 (0.00 B)

In [5]:
from src.mapper2D import no_lstm_mapper2D_model
mapper = no_lstm_mapper_model()
mapper.summary()
mapper.compile(optimizer="adam", loss="mse")

Model: "no_lstm_mapper"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_5 (InputLayer)      │ (None, 4, 32, 32, 1)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_4              │ (None, 4, 15, 15, 2)   │            20 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_5              │ (None, 4, 7, 7, 2)     │            38 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_6              │ (None, 4, 98)          │             0 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ time_distributed_7              │ (None, 4, 64)          │         6,336 │
│ (TimeDistributed)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 32)             │         8,224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 8)              │           136 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,282 (59.70 KB)

 Trainable params: 15,282 (59.70 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
#stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
#history_mapper = lstm_mapper.fit(v_train_seq, z_train, epochs=100, batch_size = 8, validation_split = 0.1, callbacks=[stop_early])
#history_mapper = lstm_mapper.fit(v_train_seq, z_train, epochs=100, batch_size = 8, validation_split = 0.1)
history_mapper = mapper.fit(v_train_seq, z_train, epochs=100, batch_size=8, validation_split=0.1)

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(history_mapper.history["loss"], label="Training Loss")
plt.plot(history_mapper.history["val_loss"], label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.tight_layout()
plt.grid(True, linestyle=':', color='gray', alpha=0.5)
plt.show()

### saving weights of trained mapper

In [ ]:
weights_dir = "weights/"
os.makedirs(weights_dir, exist_ok=True)
weights_file = os.path.join(weights_dir, "no_lstm_sim.weights.h5")
#mapper.save_weights(weights_file)
lstm_mapper.save_weights(weights_file)
print(f"Weights stored under: {weights_file}")

In [ ]:
from src.performance_eval import (
    compute_evaluation_metrics,
    plot_deviations_x_y, 
    plot_deviations_perm,
    plot_seq_recon_examples)

In [ ]:
lstm_mapper.load_weights("weights/lstm_sim.weights.h5")
#mapper.load_weights("weights/no_lstm_sim.weights.h5")
print("Weights of mapper successfully loaded.")

In [21]:
#voltage_test, gamma_test, pos = load_2Ddata("sim_data", "sim_data_set", data_type = "sim")

Loading simulation data...
Normalizing data...
Data loading complete!


In [22]:
pred_latent = lstm_mapper.predict(voltage_test)
gamma_pred_test = vae.decoder.predict(pred_latent).reshape(-1, 2840,1)

157/157 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step


In [ ]:
model_variant='sim_lstm_spiral_circle' #sim_lstm_{training_traj}_{test_traj}
n_el = 32
mesh_obj = mesh.create(n_el, h0=0.05)

plot_seq_recon_examples(mesh_obj, gamma_test, gamma_pred_test, pos, 
                               save=False, 
                               fpath='results/',
                               fname = f'seq_reconstructions_{model_variant}.pdf', num_plots=25)

In [ ]:
plot_deviations_x_y(gamma_test, 
                    gamma_pred_test, 
                    mesh_obj, 
                    save=False, 
                    fpath='results/', 
                    fname=f'deviation_plot_{model_variant}',
                    limits = (-0.2, 0.2), 
                    scale_factor = False)

In [ ]:
plot_deviations_perm(gamma_test, 
                     gamma_pred_test, 
                     mesh_obj, 
                     save=False, 
                     fpath='results/', 
                     fname=f'perm_deviation_plot_{model_variant}', 
                     binwidth=5)